This is the place where I try to understand the sklearn apis. Most of the api's seem blackboxes. Will try to derive from scratch what the output is like

In [79]:
import string
from nltk.corpus import stopwords

stopwordsSet = stopwords.words('english')
mywordDict = {}


def getTokens(line, removeStopWords):
    # Check characters to see if they are in punctuation    
    nopuncLine = "".join( [char for char in line if char not in string.punctuation])
    #conver to lower, drop spaces
    words = [x.lower() for x in nopuncLine.split(" ") if len(x) != 1 and x != "" and ((not removeStopWords) or (x.lower() not in stopwordsSet))]
    return(words)
    
    

def parseAndProcess(line, removeStopWords=False):
    if(len(line) == 0 or line == "\n"):
        return
    words = getTokens(line, removeStopWords)
    for word in words:
        val = mywordDict.get(word)
        if(not val):
            mywordDict[word] = 1
        else:
            mywordDict[word] = val + 1

In [80]:
#expt 
docs = ["Hello, I think you are such a nice person", 
         "Do you want to make thats mistake again and again",
         "Well done! errors are far off from you",
         "Hello! making errors is my birthright"]

In [81]:
#twenty_train.data[0:2]
for dat in docs:
    prsed = [parseAndProcess(line) for line in dat.split("\n")]
    
print(mywordDict)
print(len(mywordDict))

{'hello': 2, 'think': 1, 'you': 3, 'are': 2, 'such': 1, 'nice': 1, 'person': 1, 'do': 1, 'want': 1, 'to': 1, 'make': 1, 'thats': 1, 'mistake': 1, 'again': 2, 'and': 1, 'well': 1, 'done': 1, 'errors': 2, 'far': 1, 'off': 1, 'from': 1, 'making': 1, 'is': 1, 'my': 1, 'birthright': 1}
25


In [82]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(docs)
X_train_counts.shape

(4, 25)

In [83]:
print(X_train_counts.todense())
print(count_vect.vocabulary_)
print(count_vect.get_feature_names())
print(count_vect.get_params())

[[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1]
 [2 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1]
 [0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1]
 [0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0]]
{'hello': 9, 'think': 20, 'you': 24, 'are': 2, 'such': 18, 'nice': 15, 'person': 17, 'do': 4, 'want': 22, 'to': 21, 'make': 11, 'thats': 19, 'mistake': 13, 'again': 0, 'and': 1, 'well': 23, 'done': 5, 'errors': 6, 'far': 7, 'off': 16, 'from': 8, 'making': 12, 'is': 10, 'my': 14, 'birthright': 3}
['again', 'and', 'are', 'birthright', 'do', 'done', 'errors', 'far', 'from', 'hello', 'is', 'make', 'making', 'mistake', 'my', 'nice', 'off', 'person', 'such', 'thats', 'think', 'to', 'want', 'well', 'you']
{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'preprocessor': None, 'stop_words': None, 'strip_accen

In [84]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [85]:
print(tf_transformer.get_params())

{'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': False, 'use_idf': False}


In [86]:
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(4, 25)

In [87]:
X_train_tf.todense()

matrix([[ 0.        ,  0.        ,  0.37796447,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.37796447,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.37796447,  0.        ,  0.37796447,  0.37796447,  0.        ,
          0.37796447,  0.        ,  0.        ,  0.        ,  0.37796447],
        [ 0.57735027,  0.28867513,  0.        ,  0.        ,  0.28867513,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.28867513,  0.        ,  0.28867513,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.28867513,
          0.        ,  0.28867513,  0.28867513,  0.        ,  0.28867513],
        [ 0.        ,  0.        ,  0.35355339,  0.        ,  0.        ,
          0.35355339,  0.35355339,  0.35355339,  0.35355339,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.35355339,  

In [169]:
np.apply_along_axis(sum, axis=0, arr=X_train_tf.todense())

matrix([[ 0.57735027,  0.28867513,  0.73151786,  0.40824829,  0.28867513,
          0.35355339,  0.76180168,  0.35355339,  0.35355339,  0.78621276,
          0.40824829,  0.28867513,  0.40824829,  0.28867513,  0.40824829,
          0.37796447,  0.35355339,  0.37796447,  0.37796447,  0.28867513,
          0.37796447,  0.28867513,  0.28867513,  0.35355339,  1.020193  ]])

In [139]:
mywordDict.keys()

dict_keys(['hello', 'think', 'you', 'are', 'such', 'nice', 'person', 'do', 'want', 'to', 'make', 'thats', 'mistake', 'again', 'and', 'well', 'done', 'errors', 'far', 'off', 'from', 'making', 'is', 'my', 'birthright'])

In [166]:
print(docs)
#Manually calculate term frequency
import numpy as np
tf = np.zeros((len(docs),len(mywordDict)))

#Walk through all documents and put term counts in place
colIndexForWord = dict([(j,i) for i,j in enumerate(sorted(mywordDict.keys()))])
print(colIndexForWord)


for i, doc in enumerate(docs):
    #print("{}: {}".format(i,doc))
    for word in getTokens(doc,removeStopWords=False):
        tf[i,colIndexForWord[word]] += 1

#calculate IDF matrix: by browsing through terms
IDF = np.apply_along_axis(lambda x: sum([1 for e in x if e>0]), axis=0, arr=tf)

def f_idf(x):
    return(np.log10(len(docs)/x))

IDF = np.vectorize(f_idf)(IDF)  # if A is your Numpy array
print("IDF:")
print(IDF)

#calculate tf matrix
def TF(x):
    if(x <= 0.0):
        return(0.0)
    else:
        return(1.0  + np.log10(x))

TF_v = np.vectorize(TF)
tfnew = TF_v(tf)
print("tfnew: ")
print(tfnew)

#TFIDF
print("TF*IDF: ")
np.multiply(tfnew, IDF)

#DIFF
X_train_tf.todense() - np.multiply(tfnew, IDF)

['Hello, I think you are such a nice person', 'Do you want to make thats mistake again and again', 'Well done! errors are far off from you', 'Hello! making errors is my birthright']
{'again': 0, 'and': 1, 'are': 2, 'birthright': 3, 'do': 4, 'done': 5, 'errors': 6, 'far': 7, 'from': 8, 'hello': 9, 'is': 10, 'make': 11, 'making': 12, 'mistake': 13, 'my': 14, 'nice': 15, 'off': 16, 'person': 17, 'such': 18, 'thats': 19, 'think': 20, 'to': 21, 'want': 22, 'well': 23, 'you': 24}
IDF:
[ 0.60205999  0.60205999  0.30103     0.60205999  0.60205999  0.60205999
  0.30103     0.60205999  0.60205999  0.30103     0.60205999  0.60205999
  0.60205999  0.60205999  0.60205999  0.60205999  0.60205999  0.60205999
  0.60205999  0.60205999  0.60205999  0.60205999  0.60205999  0.60205999
  0.12493874]
tfnew: 
[[ 0.       0.       1.       0.       0.       0.       0.       0.       0.
   1.       0.       0.       0.       0.       0.       1.       0.       1.
   1.       0.       1.       0.       0.     

matrix([[ 0.        ,  0.        ,  0.07693448,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.07693448,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         -0.22409552,  0.        , -0.22409552, -0.22409552,  0.        ,
         -0.22409552,  0.        ,  0.        ,  0.        ,  0.25302574],
        [-0.20594784, -0.31338486,  0.        ,  0.        , -0.31338486,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.31338486,  0.        , -0.31338486,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.31338486,
          0.        , -0.31338486, -0.31338486,  0.        ,  0.1637364 ],
        [ 0.        ,  0.        ,  0.05252339,  0.        ,  0.        ,
         -0.2485066 ,  0.05252339, -0.2485066 , -0.2485066 ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.2485066 ,  

In [92]:
import numpy as np
m  = np.zeros((2,3))
m[0,1] = 10; m[0,2] = 100;
print(m)

def f(x):
    if(x<=0):
        return(1000)
    else:
        return(2*x)
f = np.vectorize(f)  # or use a different name if you want to keep the original f
result_array = f(m)  # if A is your Numpy array
print(result_array)

[[   0.   10.  100.]
 [   0.    0.    0.]]
[[1000   20  200]
 [1000 1000 1000]]


In [135]:
a = np.random.randint(100,size=28).reshape(4,7)
print(a)
b = np.ones((1,7)) * 2
c = np.ones((1,7)) * 1
b = np.concatenate((b, c, b,c), axis=0)
print(b)
np.multiply(a,b)

[[13 95 58  2 59 59 85]
 [ 0 73 74 77 60 61 17]
 [80 42 76 76 81 35 89]
 [24 46 39 86 41 60 96]]
[[ 2.  2.  2.  2.  2.  2.  2.]
 [ 1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.]
 [ 1.  1.  1.  1.  1.  1.  1.]]


array([[  26.,  190.,  116.,    4.,  118.,  118.,  170.],
       [   0.,   73.,   74.,   77.,   60.,   61.,   17.],
       [ 160.,   84.,  152.,  152.,  162.,   70.,  178.],
       [  24.,   46.,   39.,   86.,   41.,   60.,   96.]])

In [145]:
a = np.random.randint(100,size=28).reshape(4,7)

In [146]:
a

array([[88, 30, 90, 17, 57, 40, 47],
       [ 7, 99, 26,  2, 12, 63, 80],
       [79, 32, 36, 50, 76, 94, 44],
       [25, 43, 11, 45, 33, 46, 82]])

In [151]:
np.apply_along_axis(lambda x: sum([1 for e in x if e>50]), axis=0, arr=a)

array([2, 1, 1, 0, 2, 2, 2])